In [ ]:
!pip install -qU "semantic-router[pinecone]==0.0.22"

In [1]:
from semantic_router import Route

# we could use this as a guide for our chatbot to avoid political conversations
politics = Route(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the president" "don't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
    ],
)

# this could be used as an indicator to our chatbot to switch to a more
# conversational prompt
chitchat = Route(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

# we place both of our decisions together into single list
routes = [politics, chitchat]

/Users/jamesbriggs/opt/anaconda3/envs/decision-layer/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from getpass import getpass
from semantic_router.encoders import OpenAIEncoder

# get at platform.openai.com
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API key: "
)
encoder = OpenAIEncoder()

In [3]:
from semantic_router.index.pinecone import PineconeIndex

# get at app.pinecone.io
os.environ["PINECONE_API_KEY"] = os.environ.get("PINECONE_API_KEY") or getpass(
    "Enter Pinecone API key: "
)
pc_index = PineconeIndex()

In [4]:
from semantic_router.layer import RouteLayer

rl = RouteLayer(encoder=encoder, routes=routes, index=pc_index)

2024-02-14 01:40:22 INFO semantic_router.utils.logger local


We can check our route layer and index information.

In [5]:
rl.list_route_names()

['politics', 'chitchat']

In [6]:
len(rl.index)

5

We can also view all of the records for a given route:

In [7]:
rl.index._get_route_ids(route_name="politics")

{'vectors': [{'id': 'politics#1c4e5a74a2cb693bd51dd4fe0d8deb41'}, {'id': 'politics#3f95e1e2c26aa109d7a6b352bd1da93e'}, {'id': 'politics#461468866b9123c5ef111b72040c20d8'}, {'id': 'politics#e9120cb91dcc3ff76e7d2ce0740e009c'}, {'id': 'politics#f798c13e0a6d8a9926fc35a0f10f880b'}], 'pagination': {'next': 'eyJza2lwX3Bhc3QiOiJwb2xpdGljcyNmNzk4YzEzZTBhNmQ4YTk5MjZmYzM1YTBmMTBmODgwYiIsInByZWZpeCI6InBvbGl0aWNzIn0='}, 'namespace': '', 'usage': {'readUnits': 1}}
{'vectors': [], 'namespace': '', 'usage': {'readUnits': 1}}


['politics#1c4e5a74a2cb693bd51dd4fe0d8deb41',
 'politics#3f95e1e2c26aa109d7a6b352bd1da93e',
 'politics#461468866b9123c5ef111b72040c20d8',
 'politics#e9120cb91dcc3ff76e7d2ce0740e009c',
 'politics#f798c13e0a6d8a9926fc35a0f10f880b']

And query:

In [8]:
rl("don't you love politics?").name

'politics'

In [9]:
rl("how's the weather today?").name

'chitchat'

In [10]:
rl("I'm interested in learning about llama 2").name

We can delete or update routes.

In [11]:
len(rl.index)

10

In [12]:
import time

rl.delete(route_name="chitchat")
time.sleep(1)
len(rl.index)

{'vectors': [{'id': 'chitchat#1e01e2c7f411d4dfcc43fe7a3de2fb67'}, {'id': 'chitchat#34fd10a3b6d1436b49759d9c67068a7e'}, {'id': 'chitchat#74cfdf22939f052ea9be158d77bc6bda'}, {'id': 'chitchat#bc13a8bc005f437d2bdeacbb9e741b8b'}, {'id': 'chitchat#fca33e294c0c4bb08ed95e1e3fd944c0'}], 'namespace': '', 'usage': {'readUnits': 1}}


5

In [13]:
rl("how's the weather today?").name

In [14]:
rl.index.get_routes()

{'vectors': [{'id': 'politics#1c4e5a74a2cb693bd51dd4fe0d8deb41'}, {'id': 'politics#3f95e1e2c26aa109d7a6b352bd1da93e'}, {'id': 'politics#461468866b9123c5ef111b72040c20d8'}, {'id': 'politics#e9120cb91dcc3ff76e7d2ce0740e009c'}, {'id': 'politics#f798c13e0a6d8a9926fc35a0f10f880b'}], 'namespace': '', 'usage': {'readUnits': 1}}


[('politics', "they're going to destroy this country!"),
 ('politics',
  "don't you just love the presidentdon't you just hate the president"),
 ('politics', "isn't politics the best thing ever"),
 ('politics', "why don't you tell me about your political opinions"),
 ('politics', 'they will save the country!')]

In [15]:
rl.index.describe()

{'type': 'pinecone', 'dimensions': 1536, 'vectors': 5}